In [1]:
import pandas as pd
import numpy
df = pd.read_csv('BX-Book-Ratings.csv', names=["User-ID", "ISBN", "Book-Rating"], delimiter=";")
rating_matrix = numpy.ndarray(shape=(50000,50000), dtype = 'int64')
df_mat = df.as_matrix()[1:]

#print(df_mat)

C:\Users\karth\Anaconda2\envs\cs670\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
%store df_mat

Stored 'df_mat' (ndarray)


In [2]:
%store -r df_mat
totalratings = len(df_mat[:,0])
print totalratings

1149780


In [3]:
import numpy as np
df_mat[:,0] = df_mat[:,0].astype('int64')
df_mat[:,2] = df_mat[:,2].astype('int64')

usercount = (len(np.unique(df_mat[:,0])))
bookcount = (len(np.unique(df_mat[:,1])))
#numpy.unique(df_mat[:,1]).shape

In [4]:
print usercount
print bookcount

105283
340556


In [5]:
#constructing the inital rating matrix consisting of 50000 users and 50000 books

bookid = 0
usernum = 0
bookmap = {}
usermap = {}
useridslist = []
for k in range(0,800000):
    userid = df_mat[k,0]
    isbn = df_mat[k,1]
    if isbn not in bookmap:
        if bookid < 50000:
            bookmap[isbn] = bookid
            if userid not in usermap:
                if usernum < 50000:
                    usermap[userid]=usernum
                    rating_matrix[usernum, bookid] = df_mat[k,2]
                    usernum+=1
            else:
                rating_matrix[usermap[userid], bookid] = df_mat[k,2]
            bookid = bookid+1
    else:
        if userid not in usermap:
            if usernum < 50000:
                usermap[userid]=usernum
                rating_matrix[usernum, bookmap[isbn]] = df_mat[k,2]
                usernum+=1
        else:
            rating_matrix[usermap[userid], bookmap[isbn]] = df_mat[k,2]
            


In [41]:
print(rating_matrix.shape)

(50000L, 50000L)


In [42]:
import numpy as np
user_list = np.unique(df_mat[:,0])
print len(user_list)

105283


In [43]:
print len(bookmap.values())
print len(usermap.values())

#print df_mat[0,0]
#print bookmap[df_mat[0,1]]

50000
50000


In [44]:
import networkx as nx
from networkx.algorithms import bipartite

In [45]:
# constructing a bipartite graph with users on one side and items on other side

B = nx.Graph()
B.add_nodes_from(usermap.values(), bipartite=0) #0 indicates one class of nodes, here: users
B.add_nodes_from(bookmap.values(), bipartite=1) #1 for isbn

In [46]:
edge_ls = []

for k in range(0,800000):
    ls = []
    if k%50000==0:
        print k
    if df_mat[k,0] in usermap.keys() and df_mat[k,1] in bookmap.keys() and df_mat[k,2]!=0:
        ls.append(usermap[df_mat[k,0]])
        ls.append(bookmap[df_mat[k,1]])
        if df_mat[k,2] > 7:
            rate = 'L' # for love
        else: rate = 'H' # for hate
        ls.append(rate)
        edge_ls.append(ls)
        
B.add_weighted_edges_from(edge_ls)

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000


'\nls.append()\n'

In [47]:
print(len(B))

50000


In [48]:
%store -r B
import pickle
pickle.dump(B,open("a.p","wb"))

no stored variable B


In [49]:
#Building the configurations for target user- target item pair

import pickle
from collections import defaultdict
Dictionary = defaultdict(dict)
count =0
dictionary_adj = dict(B.adjacency()) 
for user1 in usermap.values():
    BFS_Dict= dict(nx.bfs_successors(B,user1))
    User_item_list = []
    User_item_list.append(user1)
    edge_list=[]
    if count%10000 ==0:
        print "Reached user ",count+1
    count+=1
    if bool(BFS_Dict):
        for book1 in BFS_Dict[user1]:
            if book1 in BFS_Dict.keys():
                edge_list.append(dictionary_adj[user1][book1]['weight'])
                for user2 in BFS_Dict[book1]:
                    edge_list.append(dictionary_adj[book1][user2]['weight'])    
                    for book2 in dictionary_adj[user2].keys(): 
                        if book2 in BFS_Dict[user1] and book2!=book1:
                            edge_list.append(dictionary_adj[user2][book2]['weight'])
                            User_item_list.append(book2)
                            Dictionary[tuple(User_item_list)][tuple(edge_list)] = Dictionary[tuple(User_item_list)].get(tuple(edge_list),0)+1
                            Dictionary[tuple(User_item_list)]["Rating"]=rating_matrix[User_item_list[0]][User_item_list[1]]
                            edge_list.pop()
                            User_item_list.pop()
                    edge_list.pop()
                edge_list.pop()
                        

Reached user  1
Reached user  10001
Reached user  20001
Reached user  30001
Reached user  40001


In [50]:
print (len(Dictionary))

161293


In [51]:
%store Dictionary
import pickle
pickle.dump(Dictionary,open("b.p","wb"))

Stored 'Dictionary' (defaultdict)


In [7]:
# naming all configurations
con = []

con.append([('H', 'H', 'H')])
con.append([('H', 'H', 'L')])
con.append([('H', 'L', 'L')])
con.append([('L', 'L', 'L')])
con.append([('L', 'L', 'H')])
con.append([('L', 'H', 'H')])
con.append([('H', 'L', 'H')])
con.append([('L', 'L', 'L')])



In [8]:
#Config matrix(config_mat) which will be our feature-vector matrix for classifiers

%store -r Dictionary
import numpy as np
config_mat = np.zeros((len(Dictionary), 9))
pair_dic = {}
i=0
for k in Dictionary.keys():
    userItemConfig = Dictionary[k]
    pair_dic[i] = k
    for key in userItemConfig.keys():
        for j in range(0,8):
            if con[j][0] == key:
                config_mat[i,j] = userItemConfig[key]
    if rating_matrix[k[0],k[1]]!=0:
        if rating_matrix[k[0],k[1]] > 7:
            config_mat[i,-1] = 1
        else: config_mat[i,-1] = 0
    #config_mat[i,-1] = rating_matrix[k[0],k[1]] 
    i+=1
            

In [54]:
%store config_mat
import pickle
pickle.dump(config_mat,open("c.p","wb"))

Stored 'config_mat' (ndarray)


In [9]:
#Multionomial NB
%store -r config_mat
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(config_mat[:,0:8], config_mat[:,-1], test_size=0.30, shuffle=False)

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
mnb_y_pred = mnb.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, mnb_y_pred))
print "RMSE with Naive Bayes:",rmse
print "Overall accuracy : ", metrics.accuracy_score(y_test,mnb_y_pred)
print "Recall score for 'Love': " , recall_score(y_test, mnb_y_pred, pos_label=1, average='binary') 
print "Recall score for 'Hate': " , recall_score(y_test, mnb_y_pred, pos_label=0, average='binary') 
print "Precision score for 'Love' : " , precision_score(y_test, mnb_y_pred, pos_label=1, average='binary') 
print "Precision score for 'Hate' : " , precision_score(y_test, mnb_y_pred, pos_label=0, average='binary') 
print "F-measure for 'Love' : " , f1_score(y_test, mnb_y_pred, pos_label=1, average='binary') 
print "F-measure for 'Hate' : " , f1_score(y_test, mnb_y_pred, pos_label=0, average='binary') 

RMSE with Naive Bayes: 0.656502313848
Overall accuracy :  0.5690047119120443
Recall score for 'Love':  0.2660914062108062
Recall score for 'Hate':  0.7811983834124056
Precision score for 'Love' :  0.46001734605377276
Precision score for 'Hate' :  0.603098377557111
F-measure for 'Love' :  0.33715793153863266
F-measure for 'Hate' :  0.6806914396827584


In [10]:
%store X_train
%store X_test
%store y_train
%store y_test

Stored 'X_train' (ndarray)
Stored 'X_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)


In [11]:
#KNN
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score


knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
KNN_y_pred = knn.predict(X_test)
rmse_knn = sqrt(mean_squared_error(y_test, KNN_y_pred))
print "RMSE with kNN:", rmse_knn
print "Overall accuracy : ", metrics.accuracy_score(y_test,KNN_y_pred)
print "Recall score for 'Love': " , recall_score(y_test, KNN_y_pred, pos_label=1, average='binary') 
print "Recall score for 'Hate': " , recall_score(y_test, KNN_y_pred, pos_label=0, average='binary') 
print "Precision score for 'Love' : " , precision_score(y_test, KNN_y_pred, pos_label=1, average='binary') 
print "Precision score for 'Hate' : " , precision_score(y_test, KNN_y_pred, pos_label=0, average='binary') 
print "F-measure for 'Love' : " , f1_score(y_test, KNN_y_pred, pos_label=1, average='binary') 
print "F-measure for 'Hate' : " , f1_score(y_test, KNN_y_pred, pos_label=0, average='binary') 

RMSE with kNN: 0.649428564607
Overall accuracy :  0.5782425394725965
Recall score for 'Love':  0.4650579441127778
Recall score for 'Hate':  0.6575294324371815
Precision score for 'Love' :  0.4875098606363397
Precision score for 'Hate' :  0.6369795390324448
F-measure for 'Love' :  0.47601930779500873
F-measure for 'Hate' :  0.6470913744206959


In [12]:
%store bookmap
%store usermap
%store pair_dic

Stored 'bookmap' (dict)
Stored 'usermap' (dict)
Stored 'pair_dic' (dict)


In [127]:
# import pandas as pd
# df_book = pd.read_csv('BX-Books.csv', names=["ISBN","Book-Title","Book-Author","Year-Of-Publication","Publisher","Image-URL-S","Image-URL-M","Image-URL-L"], delimiter=";")
# df_mat_book = df_book.as_matrix()[1:]

In [13]:
#Linear SVM
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn.svm import LinearSVC
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score


clf = LinearSVC()

clf = clf.fit(X_train, y_train)
svc_y_pred = clf.predict(X_test)
rmse_svc = sqrt(mean_squared_error(y_test, svc_y_pred))
print "RMSE with SVM:", rmse_svc
print "Overall accuracy : ", metrics.accuracy_score(y_test,svc_y_pred)
print "Recall score for 'Love': " , recall_score(y_test, svc_y_pred, pos_label=1, average='binary') 
print "Recall score for 'Hate': " , recall_score(y_test, svc_y_pred, pos_label=0, average='binary') 
print "Precision score for 'Love' : " , precision_score(y_test, svc_y_pred, pos_label=1, average='binary') 
print "Precision score for 'Hate' : " , precision_score(y_test, svc_y_pred, pos_label=0, average='binary') 
print "F-measure for 'Love' : " , f1_score(y_test, svc_y_pred, pos_label=1, average='binary') 
print "F-measure for 'Hate' : " , f1_score(y_test, svc_y_pred, pos_label=0, average='binary') 

RMSE with SVM: 0.636653569231
Overall accuracy :  0.594672232784988
Recall score for 'Love':  0.06657301961571263
Recall score for 'Hate':  0.9646107889650325
Precision score for 'Love' :  0.5685518423307626
Precision score for 'Hate' :  0.5959960046901464
F-measure for 'Love' :  0.11918983248753762
F-measure for 'Hate' :  0.7367700546242735


In [14]:
#Decision_tree
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn import tree
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
rmse_dt = sqrt(mean_squared_error(y_test, dt_y_pred))
print "RMSE with Decision tree:", rmse_dt
print "Overall accuracy : ", metrics.accuracy_score(y_test, dt_y_pred)
print "Recall score for 'Love': " , recall_score(y_test, dt_y_pred, pos_label=1, average='binary') 
print "Recall score for 'Hate': " , recall_score(y_test, dt_y_pred, pos_label=0, average='binary') 
print "Precision score for 'Love' : " , precision_score(y_test, dt_y_pred, pos_label=1, average='binary') 
print "Precision score for 'Hate' : " , precision_score(y_test, dt_y_pred, pos_label=0, average='binary') 
print "F-measure for 'Love' : " , f1_score(y_test, dt_y_pred, pos_label=1, average='binary') 
print "F-measure for 'Hate' : " , f1_score(y_test, dt_y_pred, pos_label=0, average='binary') 

RMSE with Decision tree: 0.637010537899
Overall accuracy :  0.594217574605274
Recall score for 'Love':  0.20308031906888074
Recall score for 'Hate':  0.8682129678439642
Precision score for 'Love' :  0.5191074634521672
Precision score for 'Hate' :  0.6086474501108647
F-measure for 'Love' :  0.2919476398254661
F-measure for 'Hate' :  0.7156202476645666


In [15]:
#Logistic regression
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

lr = linear_model.LogisticRegression()
lr = lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)
rmse_lr = sqrt(mean_squared_error(y_test, lr_y_pred))
print "RMSE with Logistic Regression:", rmse_lr
print "Overall accuracy : ", metrics.accuracy_score(y_test, lr_y_pred)
print "Recall score for 'Love': " , recall_score(y_test, lr_y_pred, pos_label=1, average='binary') 
print "Recall score for 'Hate': " , recall_score(y_test, lr_y_pred, pos_label=0, average='binary') 
print "Precision score for 'Love' : " , precision_score(y_test, lr_y_pred, pos_label=1, average='binary') 
print "Precision score for 'Hate' : " , precision_score(y_test, lr_y_pred, pos_label=0, average='binary') 
print "F-measure for 'Love' : " , f1_score(y_test, lr_y_pred, pos_label=1, average='binary') 
print "F-measure for 'Hate' : " , f1_score(y_test, lr_y_pred, pos_label=0, average='binary') 

RMSE with Logistic Regression: 0.636507478893
Overall accuracy :  0.5948582293130529
Recall score for 'Love':  0.05543570962725129
Recall score for 'Hate':  0.9727288701458443
Precision score for 'Love' :  0.5874534821903243
Precision score for 'Hate' :  0.5951577181929602
F-measure for 'Love' :  0.10131108462455304
F-measure for 'Hate' :  0.7384808302873456


In [7]:
#Prints top 100 books and a dictionary for top 1000 books that contains the user that have rated them, their ratings and the configuration that was formed

%store -r book_name
%store -r isbn
%store -r Dictionary
%store -r df_mat_book
Bookcount = {}

from collections import defaultdict

isbn =[row[0] for row in df_mat_book]
book_name =[row[1] for row in df_mat_book]

for i in range(len(book_name)):
    if i% 50000 == 0:
        print "Reached Book num: ", i+1
    if isbn[i] in bookmap.keys():
        if book_name[i] not in Bookcount.keys():
            Bookcount[book_name[i]]=book_name.count(book_name[i])  

top_books =[]
print ("Top Books")

counter = 0
for key,item in sorted(Bookcount.iteritems(),key=lambda (k,v):(v,k),reverse=True):
    if counter == 1000:
        break
    counter+=1
    top_books.append(key)
    if counter<100:
        print key,item
    
print("Final Dictionary")

Dict_output = defaultdict(dict)
for key, item in Dictionary.iteritems():
    isbn_temp = bookmap.keys()[bookmap.values().index(key[1])]
    if isbn_temp in isbn:
        bookname = book_name[isbn.index(isbn_temp)]
        if bookname in top_books:
            Dict_output[bookname][usermap.keys()[usermap.values().index(key[0])]]=item
 
counter=0
for key,item in Dict_output.iteritems():
    if counter==20:
        break
    if key in top_books[:20]:
        print key,item
        counter+=1
    
%store Dict_output
%store top_books
%store book_name
%store isbn

Reached Book num:  1
Reached Book num:  50001
Reached Book num:  100001
Reached Book num:  150001
Reached Book num:  200001
Reached Book num:  250001
Top Books
Selected Poems 27
Little Women 24
Wuthering Heights 21
The Secret Garden 20
Dracula 20
Adventures of Huckleberry Finn 20
Jane Eyre 19
The Night Before Christmas 18
Pride and Prejudice 18
Great Expectations 17
Masquerade 16
Frankenstein 16
Black Beauty 16
The Gift 15
Emma 15
Beloved 15
Nemesis 14
The Wedding 13
The Secret 13
Robinson Crusoe 13
Psychology 13
Legacy 13
Journey 13
Illusions 13
Fahrenheit 451 13
The Wind in the Willows 12
The Promise 12
The Hobbit 12
Sense and Sensibility 12
Sacrifice 12
Heidi 12
Hannibal 12
Dreamcatcher 12
Carrie 12
Best Friends 12
When the Wind Blows 11
The Two Towers (The Lord of the Rings, Part 2) 11
The Edge 11
Sisters 11
Secrets 11
Peter Pan 11
Paradise 11
Mirage 11
Matilda 11
King Lear 11
Homecoming 11
Cinderella 11
Ashes to Ashes 11
Whispers 10
Treasure Island 10
Titan 10
The Velveteen Rabbit

Stored 'Dict_output' (defaultdict)
Stored 'top_books' (list)
Stored 'book_name' (list)
Stored 'isbn' (list)


In [17]:
# Demo considering top 1000 books (700 books picked in random to find predicted values for predicted their user ratings and 10 of them displayed)
%store -r X_train
%store -r y_test
%store -r lr_y_pred
%store -r df_mat_book
%store -r usermap
%store -r bookmap
%store -r pair_dic
%store -r Dictionary
%store -r Dict_output
%store -r top_books
%store -r  book_name
%store  -r isbn
from random import randint

offset_train = len(X_train)
offset_test = len(y_test)

count=0
#for i in range(0,offset_test):
while(True):
    i = randint(0,offset_test-1)
    if count==700:
        break
    if lr_y_pred[i] == 0:
        lr_p = 'Hate'
    else : lr_p = 'Love'
    if y_test[i] == 0:
        yt_p = 'Hate'
    else : yt_p = 'Love'   
    if isbn.count(bookmap.keys()[bookmap.values().index(pair_dic[i+offset_train][1])])!=0:
        bookNAME = book_name[isbn.index(bookmap.keys()[bookmap.values().index(pair_dic[i+offset_train][1])])]
        userNAME = usermap.keys()[usermap.values().index(pair_dic[i+offset_train][0])]
        if bookNAME in Dict_output.keys():
            if userNAME in Dict_output[bookNAME].keys():
                    Dict_output[bookNAME][userNAME]["Predicted"] = lr_p
                    count+=1
              
count=0
for key_outer in Dict_output.keys():
    for key_inner in Dict_output[key_outer].keys():
        if "Predicted" in Dict_output[key_outer][key_inner].keys():
            if count==10:
                break
            if Dict_output[key_outer][key_inner]['Rating'] < 8:
                actual_rating = 'Hate'
            else: actual_rating = 'Love'
            print "Book ID :", key_outer, "User ID :", key_inner, "Actual Rating :",actual_rating, "Predicted Rating :", Dict_output[key_outer][key_inner]['Predicted']
            for key,item in Dict_output[key_outer][key_inner].iteritems():
                if key!= 'Rating' and key!= 'Predicted':
                    print key," :", item
            count+=1
    
%store book_name
%store isbn

Book ID : Misery User ID : 31826 Actual Rating : Love Predicted Rating : Hate
('L', 'H', 'L')  : 2
('L', 'H', 'H')  : 3
('L', 'L', 'L')  : 11
Book ID : Misery User ID : 39441 Actual Rating : Hate Predicted Rating : Hate
('L', 'H', 'H')  : 1
('L', 'L', 'L')  : 3
('L', 'L', 'H')  : 1
('H', 'H', 'L')  : 1
('H', 'L', 'L')  : 2
('L', 'H', 'L')  : 1
Book ID : Misery User ID : 42113 Actual Rating : Hate Predicted Rating : Hate
('H', 'L', 'L')  : 1
('H', 'H', 'H')  : 1
Book ID : Misery User ID : 33474 Actual Rating : Hate Predicted Rating : Hate
('L', 'H', 'L')  : 2
('H', 'H', 'H')  : 2
('H', 'L', 'H')  : 1
('H', 'L', 'L')  : 9
('H', 'H', 'L')  : 11
Book ID : Misery User ID : 6962 Actual Rating : Hate Predicted Rating : Hate
('H', 'H', 'H')  : 1
('H', 'L', 'L')  : 3
Book ID : Misery User ID : 21332 Actual Rating : Hate Predicted Rating : Hate
('L', 'L', 'L')  : 9
('L', 'H', 'H')  : 2
('L', 'H', 'L')  : 12
('L', 'L', 'H')  : 1
Book ID : Misery User ID : 78194 Actual Rating : Hate Predicted Rati

In [27]:
# Demo considering all books
%store -r X_train
%store -r y_test
%store -r lr_y_pred
%store -r df_mat_book
%store -r usermap
%store -r bookmap
%store -r pair_dic
%store -r book_name
%store -r isbn

from random import randint
isbn =[row[0] for row in df_mat_book]
book_name =[row[1] for row in df_mat_book]
count=0
# 5922 , 23938 
offset_train = len(X_train)
offset_test = len(y_test)
while(count<5):
    rnd = randint(0,offset_test-1)
    if lr_y_pred[rnd] == 0:
        lr_p = 'Hate'
    else : lr_p = 'Love'
    if y_test[rnd] == 0:
        yt_p = 'Hate'
    else : yt_p = 'Love'   
    if isbn.count(bookmap.keys()[bookmap.values().index(pair_dic[rnd+offset_train][1])])!=0:
        print " USER ID: ",usermap.keys()[usermap.values().index(pair_dic[rnd+offset_train][0])],
        print " BOOK NAME: ",book_name[isbn.index(bookmap.keys()[bookmap.values().index(pair_dic[rnd+offset_train][1])])],
        print " PREDICTED: ",lr_p," ACTUAL: ",yt_p
        count+=1


 USER ID:  26706  BOOK NAME:  Ralph S. Mouse (Cleary Reissue)  PREDICTED:  Hate  ACTUAL:  Hate
 USER ID:  62621  BOOK NAME:  Conspiracy.Com : A Novel  PREDICTED:  Love  ACTUAL:  Hate
 USER ID:  136382  BOOK NAME:  The Thorn Birds  PREDICTED:  Hate  ACTUAL:  Love
 USER ID:  49460  BOOK NAME:  Dirty Jokes and Beer : Stories of the Unrefined  PREDICTED:  Love  ACTUAL:  Love
 USER ID:  113657  BOOK NAME:  Without Reservations : The Travels of an Independent Woman  PREDICTED:  Hate  ACTUAL:  Hate
